Основная версия 5 августа 2020. Все пути захардкожены, в будущем нужно поменять.  
Версия 12 августа 2020. Написана функция download_data.

In [1]:
import os
import sys
import re
import json
import wget
import pyunpack
import fileinput
import glob
from itertools import chain
from ast import literal_eval
from tqdm import tqdm
import numpy as np
from numpy.random import choice
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers.tokenization_bert import BasicTokenizer
pd.set_option('display.max_colwidth', 500)

In [2]:
class SyntheticDatasetGenerator:
    def __init__(self, data_dir='data/'):
        self.data_dir = data_dir
        self.dataset_path = os.path.join(data_dir, 'all_data.txt')
        #self.download_data(self.data_dir)
        
        self.input_num_sentences = None
        self.output_num_sentences = None
        self.tysya_existing_words = None
        self.tsya_existing_words = None
        self.n_nn_existing_words = None        
        self.load_dics()
        self.taglist = ['REPLACE_tsya', 'REPLACE_tysya', 'REPLACE_nn, REPLACE_n', 'REPLACE_techenie', 'REPLACE_techenii']
        self.KEEP = 'O'
        self.to_file = 'data/dataset.txt'
        self.data = None
    
    def load_dics(self):
        print('Loading dictionaries...')
        
        self.techenie_existing_words = set(['течение', 'течении'])
        
        with open('data/tsya_vocab.txt', 'r') as f:
            pairs = f.read().splitlines()
        self.tysya_existing_words = set([pair.split('\t')[0] for pair in pairs])
        self.tsya_existing_words = set([pair.split('\t')[1] for pair in pairs])
        if not os.path.exists('data/all_n_nn_words.txt'):
            with open('data/n_words.txt', 'r') as f:
                n_words = f.read().splitlines()
            n_data = pd.DataFrame(n_words, columns=['text'])
            n_data['text'] = n_data['text'].str.replace('"Словоформа": "', '').str.strip('"').str.strip()
            # remove words with capital letters
            to_filter = n_data[n_data['text'].str.contains(r'[А-ЯЁ]+', regex=True)]
            #  keep those with capital letters which have a hyphen after them
            to_append = to_filter[to_filter['text'].str.contains(r'[А-ЯЁ]+-')]
            n_data = n_data[~n_data['text'].str.contains(r'[А-ЯЁ]+', regex=True)]
            n_data = n_data.append(to_append)
            nn_pattern = re.compile(r'\wнн([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b', re.IGNORECASE)
            n_pattern = re.compile(r'[аоэеиыуёюя]н([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b', re.IGNORECASE)
            nn = n_data[n_data['text'].str.contains(nn_pattern, regex=True)]
            n = n_data[n_data['text'].str.contains(n_pattern, regex=True)]
            self.n_nn_existing_words = set(nn['text']).union(set(n['text']))
            # add word writings with 'е' instead of 'ё'
            without_yo = set()
            for word in self.n_nn_existing_words:
                if word.find('ё') != -1:
                    without_yo.add(word.replace('ё', 'е'))
            self.n_nn_existing_words.update(without_yo)
            # write n_nn_words to disk for further using out of the file
            with open('data/all_n_nn_words.txt', 'w') as f:
                for line in self.n_nn_existing_words:
                    f.write(line + '\n')
        else:
            with open('data/all_n_nn_words.txt', 'r') as f:
                self.n_nn_existing_words = set(f.read().splitlines())

    def download_data(self, data_dir):
        print('Downloading data...')
        if not os.path.exists(data_dir):
            os.mkdir(data_dir)
        dataset_name = 'Magazines'
        if os.path.exists(self.dataset_path):
            raise Exception('all_data.txt already exists')
        else:
            archive_path = os.path.join(data_dir, dataset_name + '.rar')
            if not os.path.exists(archive_path):
                archive_url = 'https://linghub.ru/static/Taiga/' + dataset_name +'.rar'
                filename = wget.download(archive_url, out=archive_path)
            unrared_path = os.path.join(data_dir, dataset_name)
            if not os.path.exists(unrared_path):            
                pyunpack.Archive(archive_path).extractall(data_dir)
            file_list = sorted(glob.glob(os.path.join(unrared_path, 'texts/*.txt')))
            with open(self.dataset_path, 'w') as file:
                input_lines = fileinput.input(file_list)
                file.writelines(input_lines)

        n_words_path = os.path.join(data_dir, 'n_words.txt')
        if not os.path.exists(n_words_path):
            all_orpho_words_path = os.path.join(data_dir, 'db_word_16.06.2020.json')
            if not os.path.exists(all_orpho_words_path):
                raise FileNotFoundError('Please put db_word_16.06.2020.json in ' + os.path.abspath(data_dir))
            with open(all_orpho_words_path, 'r') as f:
                with open(n_words_path, 'w') as writefile:
                    line = f.readline()
                    while line:
                        line = f.readline()
                        if re.search('н', line) and re.search('Словоформа', line):
                            writefile.write(line)
        
        tsya_vocab_path = os.path.join(data_dir, 'tsya_vocab.txt')
        if not os.path.exists(tsya_vocab_path):
            raise FileNotFoundError('Please put tsya_vocab.txt in ' + os.path.abspath(data_dir))
        
    
    def select_data(self):
        print('Selecting lines larger than 10 symbols...')
        with open('data/all_data.txt', 'r') as f:
            texts = f.read().splitlines() # 6697245 lines
            texts = [' '.join(line.split()) for line in texts if len(line) > 10] # 5014971 lines [5304067 lines]
        return texts
    
    def sentence_tokenize(self, texts):
        print('Splitting in sentences...')
        sentences = list(chain.from_iterable(list(map(sent_tokenize, texts))))
        self.input_num_sentences = len(sentences)
        print('Input sentences before splitting on ... :', self.input_num_sentences)
        new_sentences = []
        for s in sentences:
            longer = False
            sen = re.split(r'\.\.\.\s|…\s', s)
            if len(sen) > 1:
                longer = True
            for i in range(len(sen)):
                if longer and i < len(sen) - 1 and len(sen[i]) > 1:
                    new_sentences.append(sen[i] + '...')
                elif i == len(sen) - 1 and len(sen[i]) > 1:
                    new_sentences.append(sen[i])
        self.input_num_sentences = len(new_sentences)
        print('Input sentences:', self.input_num_sentences)
        return pd.DataFrame(new_sentences, columns=['text'])
    
    def filter_characters(self, data):
        print('Filtering characters...')
        # Heuristic for correct bert tokenization
        data['text'] = data['text'].str.replace(r'[“”«»‘‛’„»«└″′]', "'", regex=True)
        symbols_to_filter = set()
        for i, sent in tqdm(enumerate(data['text'])):
            for letter in sent:
                if ord(letter) > ord('─'):
                    symbols_to_filter.add(letter)
        symbols_to_filter.update({'\u200b','\u200e', '´', '`'})
        symbols_to_filter = ''.join(sym for sym in symbols_to_filter)
        filter_regex = re.compile(r'[' + re.escape(symbols_to_filter) + r']')
        data['text'] = data['text'].str.replace(filter_regex, '', regex=True)
        return data
    
    def mark_containing_tsya_tysya(self, data):
        print('Marking tsya/tysya sentences...')
        data['tsya'] = data['text'].str.contains(r'\wтся\b', regex=True, flags=re.IGNORECASE)
        data['tysya'] = data['text'].str.contains(r'\wться\b', regex=True, flags=re.IGNORECASE)
        full_tsya_pattern = r'\b\w*тся\b'
        full_tysya_pattern = r'\b\w*ться\b'
        data['all_tsya_words'] = data[data['tsya']]['text'].str.findall(full_tsya_pattern, flags=re.IGNORECASE)
        data['all_tysya_words'] = data[data['tysya']]['text'].str.findall(full_tysya_pattern, flags=re.IGNORECASE)
        print(len(data[data['tsya']]), len(data[data['tysya']]))
        return data.drop(['tsya', 'tysya'], axis=1)

    def mark_containing_n_nn(self, data):
        print('Marking n/nn sentences...')
        nn_pattern = re.compile(r'\wнн([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b', re.IGNORECASE)
        n_pattern = re.compile(r'[аоэеиыуёюя]н([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b', re.IGNORECASE)
        data['nn'] = data['text'].str.contains(nn_pattern, regex=True)
        data['n'] = data['text'].str.contains(n_pattern, regex=True)
        full_n_pattern = r'\b\w*[аоэеиыуёюя]н(?:[аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b'
        data['all_n_words'] = data[data['n']]['text'].str.findall(full_n_pattern, flags=re.IGNORECASE)
        full_nn_pattern = r'\b\w*нн(?:[аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b'
        data['all_nn_words'] = data[data['nn']]['text'].str.findall(full_nn_pattern, flags=re.IGNORECASE)
        print(len(data[data['n']]), len(data[data['nn']]))
        return data.drop(['n', 'nn'], axis=1)
    
    def mark_containing_techenie_ii(self, data):
        print('Marking techenie/techenii sentences...')
        techenie_pattern = re.compile(r'\bтечение\b', re.IGNORECASE)
        techenii_pattern = re.compile(r'\bтечении\b', re.IGNORECASE)
        data['techenie'] = data['text'].str.contains(techenie_pattern, regex=True)
        data['techenii'] = data['text'].str.contains(techenii_pattern, regex=True)

        data['all_techenie_words'] = data[data['techenie']]['text'].str.findall(r'\bтечение\b', flags=re.IGNORECASE)
        data['all_techenii_words'] = data[data['techenii']]['text'].str.findall(r'\bтечении\b', flags=re.IGNORECASE)
        print(len(data[data['techenie']]), len(data[data['techenii']]))
        return data.drop(['techenie', 'techenii'], axis=1)
    
    def generate_one_error_type(self, data, error_name):
        print('Generating error', error_name, '...')
        if error_name == 'tsya':
            pattern_cased = re.compile(r'ТСЯ\b')
            pattern = re.compile(r'тся\b', re.IGNORECASE)
            substitute_cased = 'ТЬСЯ'
            substitute = 'ться'
            existing_words = self.tysya_existing_words
        elif error_name == 'tysya':
            pattern_cased = re.compile(r'ТЬСЯ\b')
            pattern = re.compile(r'ться\b', re.IGNORECASE)
            substitute_cased = 'ТСЯ'
            substitute = 'тся'
            existing_words = self.tsya_existing_words
        elif error_name == 'n':
            pattern_cased = re.compile(r'(?<=[аоэеиыуёюя])(?-i:Н)(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)', re.IGNORECASE)
            pattern = re.compile(r'(?<=[аоэеиыуёюя])(?-i:н)(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)', re.IGNORECASE)
            substitute_cased = 'НН'
            substitute = 'нн'
            existing_words = self.n_nn_existing_words
        elif error_name == 'nn':
            pattern_cased = re.compile(r'(?-i:НН)(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)', re.IGNORECASE)
            pattern = re.compile(r'(?-i:нн)(?=([аоыяеи]|ый|ого|ому|ом|ым|ая|ой|ую|ые|ыми|ых|ое|ою|ий|его|ему|ем|им|яя|ей|ею|юю|ие|ими|их|ее)\b)', re.IGNORECASE)
            substitute_cased = 'Н'
            substitute = 'н'
            existing_words = self.n_nn_existing_words
        elif error_name == 'techenie':
            pattern_cased = re.compile(r'\bТЕЧЕНИЕ\b', re.IGNORECASE)
            pattern = re.compile(r'\bтечение\b', re.IGNORECASE)
            substitute_cased = 'ТЕЧЕНИИ'
            substitute = 'течении'
            existing_words = self.techenie_existing_words
        elif error_name == 'techenii':
            pattern_cased = re.compile(r'\bТЕЧЕНИИ\b', re.IGNORECASE)
            pattern = re.compile(r'\bтечении\b', re.IGNORECASE)
            substitute_cased = 'ТЕЧЕНИЕ'
            substitute = 'течение'
            existing_words = self.techenie_existing_words
        original_array = []
        changed_array = []
        col_name = 'all_' + str(error_name) + '_words'
        for line in data[col_name]:
            if line is np.nan:
                changed_array.append(np.nan)
                original_array.append(np.nan)
            else:
                changed_line = []
                original_line = []
                for word in line:
                    # the order is important! first cased pattern, then pattern
                    changed_word = pattern_cased.sub(substitute_cased, word)
                    changed_word = pattern.sub(substitute, changed_word)
                    if changed_word.lower() in existing_words and changed_word != word:                        
                        changed_line.append(changed_word)
                        original_line.append(word)
                if len(changed_line) == 0:
                    changed_array.append(np.nan)
                    original_array.append(np.nan)
                else:
                    changed_array.append(changed_line)
                    original_array.append(original_line)
        data[str(error_name) + '_words'] = original_array
        data[str(error_name) + '_words_errorified'] = changed_array
        return data.drop('all_' + str(error_name) + '_words', axis=1)
    '''
    def filter_data_by_error_set(self, data):
        print(len(data[data['tsya_words_errorified'].notna()]))
        print(len(data[data['tysya_words_errorified'].notna()]))
        print(len(data[data['n_words_errorified'].notna()]))
        print(len(data[data['nn_words_errorified'].notna()]))
        print(len(data[data['techenie_words_errorified'].notna()]))
        print(len(data[data['techenii_words_errorified'].notna()]))
        print('Filtering out the sentences that cannot be errorified...')
        chosen_ids = set(data[data['n_words_errorified'].notna()].index)
        print('Data size:', len(data))
        chosen_ids.update(set(data[data['nn_words_errorified'].notna()].index))
        print('Data size:', len(data))
        chosen_ids.update(set(data[data['tsya_words_errorified'].notna()].index))
        print('Data size:', len(data))
        chosen_ids.update(set(data[data['tysya_words_errorified'].notna()].index))
        print('Data size:', len(data))
        chosen_ids.update(set(data[data['techenie_words_errorified'].notna()].index))
        print('Data size:', len(data))
        chosen_ids.update(set(data[data['techenii_words_errorified'].notna()].index))
        print('Data size:', len(data))
        self.output_num_sentences = len(chosen_ids)
        print('Output sentences:', self.output_num_sentences)
        return data.loc[chosen_ids]
    '''
    def filter_data_by_error_set(self, data):
        
        print('Filtering out the sentences that cannot be errorified...')
        potential_cols = ['tsya_words_errorified', 'tysya_words_errorified', 'n_words_errorified', 'nn_words_errorified', 
                          'techenie_words_errorified', 'techenii_words_errorified']
        
        data.dropna(subset=potential_cols, how='all', inplace=True)
        self.output_num_sentences = len(data)
        print('Output sentences:', self.output_num_sentences)
        
#         data.to_csv('data_preprocesed.csv')    
        
        return data
    
    def errorify_sentences(self, data):
        print('Errorifying sentences...')
        np.random.seed(42)
        # if more than 1 error, they may interfere with each other
        # simple setting: only 1 error
        potential_cols = ['tsya_words_errorified', 'tysya_words_errorified', 'n_words_errorified', 'nn_words_errorified', 
                          'techenie_words_errorified', 'techenii_words_errorified']
        cols = ['text'] + potential_cols + ['tsya_words', 'tysya_words', 'n_words', 'nn_words', 'techenie_words', 'techenii_words']
        potential_data = data[cols].copy()
        potential_data['error'] = np.nan
        potential_data['error_type'] = np.nan
        potential_data['error_word'] = np.nan
        potential_data.reset_index(drop=True, inplace=True)
        for i, row in tqdm(potential_data.iterrows()): # slow method, to be replaced
            #cols_maybe_errors = [col for col in potential_cols if (row[col] and row[col] is not np.nan)]
            cols_maybe_errors = [col for col in potential_cols if (row[col] and not np.all(pd.isnull(row[col])))]
            error_col = choice(cols_maybe_errors)
            # take randomly 1 possible incorrect word or none
            # change-in-future: p of 'no error' depends on the number of possible incorrect words
            potential_error_words = potential_data.loc[i, error_col]
            probabilities = np.empty(len(potential_error_words) + 1)
            probabilities[-1] = 0.25
            probabilities[:-1] = (1 - probabilities[-1]) / len(potential_error_words)
            k = choice(range(len(potential_error_words) + 1), p=probabilities)
            if k == len(potential_error_words):
                potential_data.loc[i, 'error_type'] = 'none'
                potential_data.loc[i, 'error_word'] = np.nan
                potential_data.loc[i, 'error'] = potential_data.loc[i, 'text']
            else:
                potential_data.loc[i, 'error_type'] = error_col.replace('_words_errorified', '')
                errorified = potential_data.loc[i, error_col][k]
                potential_data.loc[i, 'error_word'] = errorified
                original = potential_data.loc[i, error_col.replace('_errorified', '')][k]
                potential_data.loc[i, 'error'] = potential_data.loc[i, 'text'].replace(original, errorified)
        return potential_data[['text', 'error', 'error_word', 'error_type']]
    
    def tokenize(self, sentence):
        return BasicTokenizer(do_lower_case=False).tokenize(sentence)

    def tag_in_conll_format(self, data):
        data['tag'] = 'REPLACE_' + data['error_type']
        exceptions = dict()
        column_names = list(data.columns)
        fast_data = data.values.tolist()
        tokens = []
        labels = []
        for i in tqdm(range(len(fast_data))):
            row = fast_data[i]
            error_col_id = column_names.index('error')
            error_word_col_id = column_names.index('error_word')                
            tag_col_id = column_names.index('tag')
            sent_tokens = self.tokenize(row[error_col_id])
            sent_labels = [self.KEEP] * len(sent_tokens)
            if not pd.isnull(row[error_word_col_id]):
                error_word = str(row[error_word_col_id])
                try:
                    error_word_id = sent_tokens.index(error_word)
                    sent_labels[error_word_id] = row[tag_col_id]
                except Exception as e:
                    print(i, e)
                    exceptions[i] = row
            tokens.append(sent_tokens)
            labels.append(sent_labels)
        self.output_num_sentences = len(tokens)
        return tokens, labels, exceptions
    
    def write_dataset_to_file(self, tokens, labels):
        with open(self.to_file, 'w') as f:
            zipped = list(zip(tokens, labels))
            for i in range(len(zipped)):
                sentence_tokens = zipped[i][0]
                sentence_labels = zipped[i][1]
                for j in range(len(sentence_tokens)):
                    f.write(sentence_tokens[j] + '\t' + sentence_labels[j] + '\n')
                f.write('\n')
    
    def generate(self, to_file=None):
#         if to_file is not None:
#             self.to_file = to_file
#         self.data = self.select_data()
#         print('Data size:', len(self.data))
# #         with open('data/all_data_longer_than_10.txt') as f:
# #             self.data = f.read().splitlines()
#         print('Data size:', len(self.data))
#         self.data = self.sentence_tokenize(self.data)
#         print('Data size:', len(self.data))
#         self.data = self.filter_characters(self.data)
#         print('Data size:', len(self.data))
#         self.data.to_csv('prepared_data.csv')
        self.data = pd.read_csv('prepared_data.csv', index_col=0)
        print(self.data.info(memory_usage="deep"))
        self.data = self.mark_containing_tsya_tysya(self.data)
        print('Data size:', len(self.data))
        self.data = self.mark_containing_n_nn(self.data)
        print('Data size:', len(self.data))
        self.data = self.mark_containing_techenie_ii(self.data)
        print('Data size:', len(self.data))
        for error_name in ['tsya', 'tysya', 'n', 'nn', 'techenie', 'techenii']:
            self.data = self.generate_one_error_type(self.data, error_name)
            print('Data size:', len(self.data))
        self.data = self.filter_data_by_error_set(self.data)
        print('Data size:', len(self.data))
        self.data = self.errorify_sentences(self.data)
        print('Data size:', len(self.data))
        tokens, labels, exceptions = self.tag_in_conll_format(self.data)
        print('Data size:', len(tokens))
        self.data = self.data.drop(exceptions.keys())
        self.write_dataset_to_file(tokens, labels)
        return self.data, tokens, labels

In [3]:
# data = pd.read_csv('data_preprocesed.csv', index_col=0)
# data = data.reset_index(drop=True)

In [4]:
%%time
generator = SyntheticDatasetGenerator()

Loading dictionaries...
CPU times: user 79.5 ms, sys: 9.57 ms, total: 89.1 ms
Wall time: 235 ms


In [5]:
%%time
generated_data, tokens, labels = generator.generate(to_file='data/dataset_plus_correct.txt')
# Input sentences should be 15664256 before sentence_tokenize function: correct
# Input sentences should be 16135506: correct
# Output sentences should be 1057067: correct

/home/kristina/IndustrialImmersion/venv_kris/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16135506 entries, 0 to 16135505
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   text    object
dtypes: object(1)
memory usage: 3.7 GB
None
Marking tsya/tysya sentences...
1306636 593751
Data size: 16135506
Marking n/nn sentences...


/home/kristina/IndustrialImmersion/venv_kris/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


4597277 2599852
Data size: 16135506
Marking techenie/techenii sentences...
13796 484
Data size: 16135506
Generating error tsya ...
Data size: 16135506
Generating error tysya ...
Data size: 16135506
Generating error n ...
Data size: 16135506
Generating error nn ...
Data size: 16135506
Generating error techenie ...
Data size: 16135506
Generating error techenii ...
Data size: 16135506
Filtering out the sentences that cannot be errorified...
Output sentences: 1069464
Data size: 1069464
Errorifying sentences...


1069464it [12:53, 1382.34it/s]


Data size: 1069464


 57%|█████▋    | 612449/1069464 [01:35<01:02, 7347.32it/s]

611106 'манны' is not in list


100%|██████████| 1069464/1069464 [02:46<00:00, 6421.67it/s]


Data size: 1069464
CPU times: user 21min 58s, sys: 19.1 s, total: 22min 17s
Wall time: 22min 13s


In [2]:
archive_url = 'http://bit.ly/2pvhWZm'
filename = wget.download(archive_url, os.path.join('./data', 'news' + '.zip'))

HTTPError: HTTP Error 503: Service Temporarily Unavailable

In [ ]:
pyunpack.Archive('./data/news.zip').extractall('./data')
# file_list = sorted(glob.glob(os.path.join(unrared_path, 'texts/*.txt')))
# with open(self.dataset_path, 'w') as file:
#     input_lines = fileinput.input(file_list)
#     file.writelines(input_lines)

In [28]:
# import nltk
# nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/kristina/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
generated_data.shape

(1069463, 5)

In [7]:
generated_data['error_type'].value_counts()

n           307414
none        267100
tysya       184171
tsya        178970
nn          121851
techenie      9633
techenii       324
Name: error_type, dtype: int64

In [8]:
(generated_data['error_type'] == 'none').sum() / len(generated_data)

0.2497515108049554

In [9]:
generated_data

,text,error,error_word,error_type,tag
0,"В недавней вашей прозе о Слуцком* вы так описываете его поэтику: 'хриплое клокотание'; 'страшная и обыденная жизнь и смерть, страшно обыденная и обыденно страшная'; 'разговорная речь с вкрапленьями профессионального и бытового жаргона'; 'небрежный (как бы) или иронический тон высказывания и тут же рядом — речь ораторская, поддержанная высокой архаикой вплоть до церковнославянизмов'; 'речевой сплав горнего и дольнего' — и далее, как бы итожа: 'преткновенная гармония', 'лиро-эпос'...","В недавней вашей прозе о Слуцком* вы так описываете его поэтику: 'хриплое клокотание'; 'страшная и обыденная жизнь и смерть, страшно обыденная и обыденно страшная'; 'разговорная речь с вкрапленьями профессионального и бытового жаргонна'; 'небрежный (как бы) или иронический тон высказывания и тут же рядом — речь ораторская, поддержанная высокой архаикой вплоть до церковнославянизмов'; 'речевой сплав горнего и дольнего' — и далее, как бы итожа: 'преткновенная гармония', 'лиро-эпос'...",жаргонна,n,REPLACE_n
1,"Когда Лермонтов написал 'Есть речи — значенье Темно иль ничтожно...' — он-то написал это не ничтожно, а осмысленно.","Когда Лермонтов написал 'Есть речи — значенье Темно иль ничтожно...' — он-то написал это не ничтожно, а осмысленно.",NaN,none,REPLACE_none
2,"О. Ч. К любым языковым формулам надо относиться осторожно, хотя и сказано энергично.","О. Ч. К любым языковым формулам надо относится осторожно, хотя и сказано энергично.",относится,tysya,REPLACE_tysya
3,"Для кого-то так относиться к замыслу важно — тому же Бродскому, который писал достаточно протяженные, синтаксически развернутые вещи.","Для кого-то так относится к замыслу важно — тому же Бродскому, который писал достаточно протяженные, синтаксически развернутые вещи.",относится,tysya,REPLACE_tysya
4,"Картина хороша, но это жанр.","Картинна хороша, но это жанр.",Картинна,n,REPLACE_n
...,...,...,...,...,...
1069459,"Душно в комнате было, и на стене картина, как форточка, открывалась,","Душно в комнате было, и на стене картина, как форточка, открывалась,",NaN,none,REPLACE_none
1069460,"Посмотри – все дробится в его узоре, и на самой границе небес и вод обнажается детский затылок моря, закругляется горизонт.","Посмотри – все дробится в его узоре, и на самой границе небес и вод обнажается детский затылок моря, закругляется горизонт.",NaN,none,REPLACE_none
1069461,"Иногда асфальт становится на дыбы, корни и грибы растут из него.","Иногда асфальт становиться на дыбы, корни и грибы растут из него.",становиться,tsya,REPLACE_tsya
1069462,Вымерзают глубины до дна.,Вымерзают глубинны до дна.,глубинны,n,REPLACE_n


## Todo

### Ошибки, которые мы взяли в работу

1. Заменить a на b:
    - -ться <-> -тся: просто ищем тся **[РЕАЛИЗОВАНО]**
    - н <-> нн (совершенно, длина): н/нн + список окончаний (см. ниже) **[РЕАЛИЗОВАНО]**
    - течение <-> течении: просто ищем
    - ни- <-> не-: не-ни в начале слова
2. Вставить/убрать пробел: имеет ли смысл в рандомных местах? Наверное, не очень пока
    - ни * <-> ни* (никто)
    - то же <-> тоже
    - что бы <-> чтобы (и чтоб)
    - бестолку <-> бес толку
    - (сложные слова) мега масштабы <-> мегамасштабы
3. Вставить/убрать дефис:
    - из-за <-> из за
    - *-то <-> * то
    - *-таки <-> * таки
4. Комбинация (1) и (2):
    - не кто <-> никто

## Ошибки, которые мы пока не взяли в работу:
1. Опечатки: вставить/заменить букву
    - ве(ч)ером, в(а)ши, бес колебаний, белее того, без везти, ...
2. Вставить предлог
    - письмо (с) фотографиями, ...

### Tagging scheme

#### Replace operations

- replace_tsya
- replace_tysya
- replace_n
- replace_nn
- replace_techenii
- replace_techenie
- replace_ni
- replace_ne

#### Merge and Split operations

- merge: ни кто -> никто
- split: не весел -> невесел
- merge_hyphen: из за -> из-за
- split_hyphen: я-таки -> я таки
- merge_replace_ni: не кто -> никто
- merge_replace_ne:
- split_replace_ni:
- split_replace_ne: 

### Списки окончаний

1. **Список окончаний прилагательных и причастий с н/нн**:  
-ый, -ого, -ому, -ом, -ым, -ая, -ой, -ую, -ые, -ыми, -ых, -а, -о, -ы  
На всякий случай мягкий вариант (лингвисты сказали не делать, но, может, всё-таки добавить на всякий случай?):  
ий|его|ему|ем|им|яя|ей|юю|ие|ими|их|я|е|и  
и пустое окончание, но нам оно не нужно, потому что там гласный вставляется: "пустынен"